In [ ]:
#!pip install transformers torch

In [ ]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer


model_id = 'naver/splade_v2_max'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [ ]:
text = "Information Retrieval course at the University of Southern Maine"
tokens = tokenizer(text, return_tensors='pt')
output = model(**tokens)
print(output)
print(output.logits.shape)
import torch

vec = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens.attention_mask.unsqueeze(-1),
dim=1)[0].squeeze()
# Get SPLADE Vector
print(vec.shape) #torch.Size([30522])


In [ ]:
# extract non-zero positions
cols = vec.nonzero().squeeze().cpu().tolist()
print(len(cols))

# extract the non-zero values
weights = vec[cols].cpu().tolist()
# use to create a dictionary of token ID to weight
sparse_dict = dict(zip(cols, weights))
sparse_dict

# extract the ID position to text token mappings
idx2token = {
    idx: token for token, idx in tokenizer.get_vocab().items()
}


In [ ]:
# map token IDs to human-readable tokens
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens


In [ ]:
import torch

text1 = "Information Retrieval course at the University of Southern Maine"
text2 = "Courses about search engines in Maine"
text3 = "Courses to avoid in computer science"
tokens1 = tokenizer(text1, return_tensors='pt')
tokens2 = tokenizer(text2, return_tensors='pt')
tokens3 = tokenizer(text3, return_tensors='pt')

output = model(**tokens1)
vec1 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens1.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

output = model(**tokens2)
vec2 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens2.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

output = model(**tokens3)
vec3 = torch.max(torch.log(1 + torch.relu(output.logits)
    ) * tokens3.attention_mask.unsqueeze(-1),dim=1)[0].squeeze().view(-1, 1)

cos = torch.nn.CosineSimilarity(dim=0)
output = cos(vec1, vec2)
print(output)
output = cos(vec1, vec3)
print(output)